<br>
# **RNN 과 LSTM** 
번역과 챗봇 모델의 기본

## **CNN**
주로 이미지 분석에서 특징을 추출하는데 활용

<br></br>

## **RNN**
자연어, 음성데이터와 같이 순서가 있는 데이터를 처리하는데 적합한 신경망

<br></br>
## **1 Basic-RNN을 사용한 손글씨 인식모델 학습하기**
1. MNIST 데이터를 RNN 학습한다
1. 회귀식, CNN과 비교해서 다양한 위상값의 모델이 생성된다

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 1. 입력값 및 파라미터를 정의
learning_rate = 0.001
total_epoch = 20
batch_size = 512

# RNN 은 순서가 있는 (ex)시계열) 자료를 다루는 모델이다
n_input = 28  # n_input (W_hh) : 가로 픽셀수로 한 번에 입력받는 갯수 
n_step  = 28  # n_step  (W_hy) : 세로 픽셀수로 총 몇 단계 데이터를 받을지를 설정
n_hidden, n_class = 128, 10

In [3]:
# 2. 신경망 모델 구성
# (1) 모델 매개변수 설정
tf.reset_default_graph()                                 # tf 매개변수를 초기화 
X = tf.placeholder(tf.float32, [None, n_step, n_input])  # X 파라미터 3개 입력가능
Y = tf.placeholder(tf.float32, [None, n_class])          

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))
# BasicRNNCell, BasicLSTMCell, GRUCell : 이 함수들을 사용하면, 구조변경에 용이하다
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

In [4]:
# (2) Graph 모델의 설정
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# outputs : 학습한 결과값이, Target 인 Y값과 동일한 [batch_size, n_class] 크기여야 한다
# 이를 위해서 Hidden Layer 의 출력은, 가중치의 W와 같은 형태로 만들어야 행렬 곱 수행이 가능하다
# n_step, None(Batch) 의 순서를 바꾸고
outputs = tf.transpose(outputs, [1, 0, 2])
# 마지막 인덱스에 위치한 n_step 차원을 제거하여, 마지막 결과값만 취한다
outputs = outputs[-1]
# 신경망 기본 모델을 정의한다
model   = tf.matmul(outputs, W) + b    

In [5]:
# (3) 손실함수(레이어 연산결과 판단값을 출력)와 최적화함수(오차를 보정한 재학습의 초기값을 출력)를 정의
# cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [6]:
# 3. 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)

In [7]:
for epoch in range(total_epoch):
    total_cost = 0

    for i in range(total_batch):
        # batch 크기에 맞게 데이터를 불러온다
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # X를 RNN 입력 크기에 맞게 [batch_size, n_step, n_input] 형태로 batch 데이터를 변환
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    if epoch % 3 == 0 :
        print('Epoch:{:4d}  Avg. cost = {:.3f}'.format( (epoch + 1), (total_cost / total_batch)))
print('최적화 완료!')

Epoch:   1  Avg. cost = 1.004
Epoch:   4  Avg. cost = 0.210
Epoch:   7  Avg. cost = 0.140
Epoch:  10  Avg. cost = 0.113
Epoch:  13  Avg. cost = 0.098
Epoch:  16  Avg. cost = 0.084
Epoch:  19  Avg. cost = 0.072
최적화 완료!


In [8]:
# 4. 결과 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy   = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.9719


<br></br>
## **2 LSTM - RNN을 사용한 단어 자동완성 모델 학습하기**
1. 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성한다.
1. Basic RNN과 비교하여, Cell 거리에 따른 비중을 반영하여 (tan h) 모델을 생성한다

In [66]:
import tensorflow as tf
import numpy as np
# 알파벳 목록을 One-Hot 인코딩 배열생성 : {'a':0, 'b':1, 'c':2, ... ,}
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [67]:
# 알파벳 배열을 인덱스 번호로 변환 : input_batch, target_batch 에 적용 
# [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
# np.eye() 대각성분은 '1', 나머지 '0'인 항등행렬/ 단위행렬을 생성한다
def make_batch(seq_data):
    input_batch, target_batch = [], []
    # 단어의 알파벳을 one-hot을 batch 데이터 생성
    for seq in seq_data: 
        input_num = [num_dic[n] for n in seq[:-1]]  # 3, 3, 15, 4, 3 ...
        target    = num_dic[seq[-1]]                
        input_batch.append(np.eye(dic_len)[input_num]) # 출력 1 : one-hot 인코딩
        target_batch.append(target)                    # 출력 2 : one-hot 인덱스값
    return input_batch, target_batch

In [68]:
# 1. RNN 신경망 모델 정의
# (1) 파라미터 설정 : RNN 시퀀스 갯수  ex) 타입 스텝: [1 2 3] => 3
tf.reset_default_graph()          # tf 매개변수 초기화 함수 
learning_rate = 0.01
n_step = 3                        # 전체 단어중 처음 3글자를 단계적 학습
n_hidden, total_epoch = 64, 30
n_input = n_class = dic_len       # 출력, 입력값 모두 26 (알파벳 one-hot을 사용)

In [69]:
# (2) 알파벳 one-hot 인코딩으로 26개 값으로 출력
# cost함수 : sparse_softmax_cross_entropy_with_logits 는 인덱스를 그대로 사용
# if) one-hot 인코딩 사용 시 : ( tf.float23, [None, n_class] )
# 1번 학습시 batch 에 해당하는 크기의 데이터를 사용한다 
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32,   [None])  # one-hot 의 인덱스데이터 사용 ex) [2], [12] ...

W = tf.Variable(tf.random_normal([n_hidden, n_class])) #, reuse=True)
b = tf.Variable(tf.random_normal([n_class]))           #, reuse=True)  #https://github.com/tensorflow/tensorflow/issues/799

In [70]:
# (3) RNN 셀의 정의
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합을 방지하기 위하여 Dropout을 설정
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) # Cell1, Cell2를 조합한 RNN 셀

In [71]:
# (4) 최종 결과는 one-hot 인코딩 형식으로 변환
# tf.nn.dynamic_rnn 를 이용해 순환 신경망(Deep RNN)을 생성 (time_major = True)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model   = tf.matmul(outputs, W) + b

In [72]:
# softmax_cross_entropy_with_logits : 텍스트 데이터 one-hot의 index 를 받는다
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# Train 데이터를 정의한다
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [73]:
# 2. RNN 신경망 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch % 3 == 0:
        print('Epoch: {:.4f} cost = {:.6f}'.format(epoch + 1, loss))
print('최적화 완료!')

Epoch: 1.0000 cost = 3.248800
Epoch: 4.0000 cost = 1.919196
Epoch: 7.0000 cost = 0.800599
Epoch: 10.0000 cost = 0.524708
Epoch: 13.0000 cost = 0.752066
Epoch: 16.0000 cost = 0.287689
Epoch: 19.0000 cost = 0.306339
Epoch: 22.0000 cost = 0.412427
Epoch: 25.0000 cost = 0.211083
Epoch: 28.0000 cost = 0.107284
최적화 완료!


In [74]:
# 3. 모델의 성능을 평가 (레이블이 one-hot 인덱스 정수이므로, 예측값도 정수로 변경)
# tf.cast(x, dtype, name=None) : 텐서를 새로운 자료형으로 변환
prediction       = tf.cast(tf.argmax(model, 1), tf.int32) # one-hot의 인덱스 추출
prediction_check = tf.equal(prediction, Y)       # 입력값을 그대로 비교한다
accuracy         = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

In [75]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:-1] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)
sess.close()


=== 예측 결과 ===
입력값: ['w ', 'w ', 'd ', 'd ', 'c ', 'c ', 'l ', 'l ', 'k ', 'k ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


<br></br>
## **3 Seq 2 Seq : 구글의 기계번역에 사용하는 신경망 모델**
1. 입력 RNN과 출력 신경망의 조합한 모델로써, 입력된 문장과 출력된 문장간의 학습시 사용된다
1. LSTM을 encoder 와 Decode로 분리하여 단위 뉴런의 길이가 더 길고 위상값이 서로다른 다른 모델을 학습한다
1. [챗봇 블로그](http://yujuwon.tistory.com/entry/TENSORFLOW-seq2seq-%EA%B8%B0%EB%B0%98-%EC%B1%97%EB%B4%87-%EB%A7%8C%EB%93%A4%EA%B8%B0)

<img src="https://t1.daumcdn.net/cfile/tistory/24FA50335965BB032F?original" align="left" width="800">

In [83]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [84]:
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd'] ||  to   -> ['t', 'o', 'P', 'P']
import tensorflow as tf
import numpy as np

# 1. Input Data를 설정한다
# (1) elements 를 정의한 뒤 (알파벳과 한글), {dict} 데이터 형식으로 변환
char_arr = [ c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑'] 
num_dic  = { n : i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

In [85]:
# (2) 번역학습 데이터 입력
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

# (3) 데이터를 One-Hot-Encoding 으로 변환
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]           # 인코더 셀 입력값 : 입력단어를 한글자씩 떼어 배열생성
        output = [num_dic[n] for n in ('S' + seq[1])]  # 디코더 셀 입력값 : 시작을 나타내는 'S'심볼을 맨 앞에 붙인다
        target = [num_dic[n] for n in (seq[1] + 'E')]  # 디코더 셀 출력값 : 비교값으로, 끝을 알리기 위해 끝에 'E'를 붙인다
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아니다 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)
    return input_batch, output_batch, target_batch

In [86]:
# 2. 모델의 정의
# (1) 파라미터값 설정
tf.reset_default_graph()      # tf 매개변수를 초기화 
learning_rate = 0.01
n_hidden, total_epoch = 128, 100
n_class = n_input = dic_len  # 입력과 출력 : one-hot 인코딩으로 크기가 같다

# (2) 인코더/ 디코더 입 출력 설정
# Seq2Seq은 '인코더 입력'과 '디코더 입력'의 형식이 같다 : [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])  # [batch size, time steps]

In [87]:
# (3) 인코더/ 디코더 셀 설정
# 1) tf.variable_scope() : 인코더 셀을 정의한다
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = 0.5) 
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)

In [88]:
# 2) 디코더 셀을 정의한다
# '인코더 셀'의 '최종 상태값'을, '디코더 셀'의 '초기 상태값'과 연결한다
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, 
                                            dtype = tf.float32)

In [89]:
# (4) 설정한 매개변수와 Cell을 연결하여 Seq2Seq 모델을 생성한다
model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [90]:
# 3. 모델의 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 5 == 0 :
        print('Epoch: {:4d}  cost = {:.6f}'.format((epoch + 1),loss))
print('최적화 완료!')

Epoch:    1  cost = 3.789763
Epoch:    6  cost = 0.275871
Epoch:   11  cost = 0.171605
Epoch:   16  cost = 0.067686
Epoch:   21  cost = 0.025324
Epoch:   26  cost = 0.007600
Epoch:   31  cost = 0.002211
Epoch:   36  cost = 0.005184
Epoch:   41  cost = 0.001348
Epoch:   46  cost = 0.000941
Epoch:   51  cost = 0.024831
Epoch:   56  cost = 0.001072
Epoch:   61  cost = 0.000815
Epoch:   66  cost = 0.000429
Epoch:   71  cost = 0.001309
Epoch:   76  cost = 0.000507
Epoch:   81  cost = 0.001037
Epoch:   86  cost = 0.000600
Epoch:   91  cost = 0.000123
Epoch:   96  cost = 0.000368
최적화 완료!


In [91]:
# 4. 모델의 평가 (번역 테스트) : 입력받은 단어의 번역을 예측하고 디코딩하는 함수
# 영한 번역 데이터의 입력
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

In [92]:
# (1) 예측시 사전에 없는 단어는 'P'로 채운다. ex) ['word', 'PPPP']
# (2) [batch size, time step, input] 를 예측결과로 출력 : 2번째 차원값 input차원의 argmax로 예측한다
def translate(word):
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]  # 결과 값 '인덱스'에 해당글자를 사용하여 배열을 만든다.
    end = decoded.index('E')                    # 출력끝 을 의미하는 'E' 이후의 글자들은 제거한다
    translated = ''.join(decoded[:end])
    return translated

print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 사랑


<br>
1. seq2seq 챗봇 대화내용 샘플
1. 모델의 평가는 좋지만 2년전 데이터라 최신과 대응성에 의문
## https://github.com/suriyadeepan/practical_seq2seq
## http://yujuwon.tistory.com/entry/TENSORFLOW-seq2seq-%EA%B8%B0%EB%B0%98-%EC%B1%97%EB%B4%87-%EB%A7%8C%EB%93%A4%EA%B8%B0